In [1]:
import keras
from keras import callbacks
from keras.datasets import mnist
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
RN_input = np.load('RN_input.npy')
a = np.load('a.npy')
print (RN_input.shape)
print (a.shape)

(9995, 190, 1536)
(9995,)


In [3]:
RN_input_small = RN_input[0:1000]
a_small = a[0:1000]
print (RN_input_small.shape)
print (a_small.shape)

(1000, 190, 1536)
(1000,)


In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(a_small)
train_labels = le.transform(a_small)

In [5]:
np.unique(train_labels)
l = len(np.unique(train_labels))

In [6]:
one_hot_labels = keras.utils.to_categorical(train_labels, num_classes=l)
print (one_hot_labels)

[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]


In [9]:
batch_size = 1
factor = 1
input_shape = RN_input_small.shape[1:3]
g_units = [256,256,256,256]*factor

inputs = Input((input_shape))
g1 = Dense(g_units[0], activation = 'relu')(inputs)
g2 = Dense(g_units[1], activation = 'relu')(g1)
g3 = Dense(g_units[2], activation = 'relu')(g2)
g4 = Dense(g_units[3], activation = 'relu')(g3)
# g_out = Reshape((-1, batch_size, g_units[3]))(g4)
g_sum = Lambda(lambda x: K.sum(x, axis=1))(g4)


f_units = [256*factor,512*factor,l] # for first 10 samples of the RN_input

f1 = Dense(f_units[0], activation = 'relu')(g_sum)
f2 = Dense(f_units[1], activation = 'relu')(f1)
f3 = Dense(f_units[2], activation = 'softmax')(f2)  

In [10]:
model = Model(input = inputs, output = f3)

/home/nvaitc/anaconda3/envs/rn/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 190, 1536)         0         
_________________________________________________________________
dense_1 (Dense)              (None, 190, 256)          393472    
_________________________________________________________________
dense_2 (Dense)              (None, 190, 256)          65792     
_________________________________________________________________
dense_3 (Dense)              (None, 190, 256)          65792     
_________________________________________________________________
dense_4 (Dense)              (None, 190, 256)          65792     
_________________________________________________________________
lambda_1 (Lambda)            (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
__________

In [12]:
model.compile(optimizer = Adam(lr = 2e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [13]:
# model_checkpoints = callbacks.ModelCheckpoint("weights_{epoch:02d}_{val_loss:.2f}.h5", monitor='val_loss',
#                                                     verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
model.fit(x = RN_input_small, 
          y = one_hot_labels, 
          epochs=200, 
          validation_split= 0.1, 
#           callbacks=[model_checkpoints]
         )

Train on 900 samples, validate on 100 samples
Epoch 1/200
900/900 [==============================] - 6s 7ms/step - loss: 1.8431 - acc: 0.2133 - val_loss: 1.8043 - val_acc: 0.1700
Epoch 2/200
900/900 [==============================] - 1s 1ms/step - loss: 1.7737 - acc: 0.2344 - val_loss: 1.8463 - val_acc: 0.1900
Epoch 3/200
900/900 [==============================] - 1s 2ms/step - loss: 1.7479 - acc: 0.2444 - val_loss: 1.7746 - val_acc: 0.2100
Epoch 4/200
900/900 [==============================] - 1s 1ms/step - loss: 1.7229 - acc: 0.2767 - val_loss: 1.8397 - val_acc: 0.1800
Epoch 5/200
900/900 [==============================] - 1s 1ms/step - loss: 1.7387 - acc: 0.2456 - val_loss: 1.8954 - val_acc: 0.1200
Epoch 6/200
900/900 [==============================] - 1s 2ms/step - loss: 1.6893 - acc: 0.2822 - val_loss: 1.9079 - val_acc: 0.1700
Epoch 7/200
900/900 [==============================] - 1s 1ms/step - loss: 1.6793 - acc: 0.3100 - val_loss: 1.8672 - val_acc: 0.1700
Epoch 8/200
900/900 [==